In [1]:
!pip install google-generativeai requests


In [ ]:
# ====== CONFIG ======

GITHUB_TOKEN = "my token"
GEMINI_API_KEY = "my key"

GITHUB_USERNAME = "NadineMlayeh"
REPOSITORY_NAME = "ai-math-utils"
PYTHON_FILE_PATH = "math_utils.py"

import requests

headers = {
    "Authorization": f"token {GITHUB_TOKEN}",
    "Accept": "application/vnd.github.v3.raw"
}

url = f"https://api.github.com/repos/{GITHUB_USERNAME}/{REPOSITORY_NAME}/contents/{PYTHON_FILE_PATH}"

response = requests.get(url, headers=headers)

if response.status_code != 200:
    raise Exception("Impossible de lire le fichier depuis GitHub")

repo_code = response.text

print("=== Code récupéré depuis GitHub ===\n")
print(repo_code)



=== Code récupéré depuis GitHub ===

def add(a, b):
    """Return the sum of two numbers"""
    return a + b


def subtract(a, b):
    """Return the difference of two numbers"""
    return a - b


def multiply(a, b):
    """Return the product of two numbers"""
    return a * b


def divide(a, b):
    """Return the division of a by b"""
    if b == 0:
        raise ValueError("Cannot divide by zero")
    return a / b



In [19]:
# =========================
# FETCH FILE FROM GITHUB
# =========================
import requests
github_url = f"https://api.github.com/repos/{GITHUB_USERNAME}/{REPOSITORY_NAME}/contents/{PYTHON_FILE_PATH}"

headers = {
    "Authorization": f"token {GITHUB_TOKEN}",
    "Accept": "application/vnd.github.v3.raw"
}

response = requests.get(github_url, headers=headers)

if response.status_code != 200:
    raise Exception("❌ Failed to fetch file from GitHub")

repo_code = response.text
print(repo_code)

# =========================
# GEMINI CONFIGURATION
# =========================
import google.generativeai as genai
genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel("gemini-2.5-flash")



def add(a, b):
    """Return the sum of two numbers"""
    return a + b


def subtract(a, b):
    """Return the difference of two numbers"""
    return a - b


def multiply(a, b):
    """Return the product of two numbers"""
    return a * b


def divide(a, b):
    """Return the division of a by b"""
    if b == 0:
        raise ValueError("Cannot divide by zero")
    return a / b



In [20]:
prompt = f"""
You are a senior Python QA engineer.

Given this Python file, generate:
1. A pytest test file covering all functions and edge cases
2. A README.md explaining usage and how to run tests

Rules:
- Use pytest
- Output ONLY valid JSON
- No markdown
- No explanations

JSON format:
{{
  "test_file_name": "test_math_utils.py",
  "test_file_content": "...",
  "readme_content": "..."
}}

Python code:
{repo_code}
"""



In [24]:

# =========================
# CALL GEMINI
# =========================
response = model.generate_content(prompt)

print("\n==============================")
print("🤖 AI GENERATED FILES")
print("==============================\n")
print(response.text)


🤖 AI GENERATED FILES

```json
{
  "test_file_name": "test_math_utils.py",
  "test_file_content": "import pytest\nfrom math_utils import add, subtract, multiply, divide # Assuming functions are in math_utils.py\n\n\n# Test cases for add function\ndef test_add_positive_integers():\n    assert add(2, 3) == 5\n\ndef test_add_negative_integers():\n    assert add(-2, -3) == -5\n\ndef test_add_mixed_integers():\n    assert add(5, -3) == 2\n    assert add(-5, 3) == -2\n\ndef test_add_with_zero():\n    assert add(0, 5) == 5\n    assert add(5, 0) == 5\n    assert add(0, 0) == 0\n\ndef test_add_float_numbers():\n    assert add(2.5, 3.5) == 6.0\n    assert add(1.1, 2.2) == pytest.approx(3.3)\n\ndef test_add_large_numbers():\n    assert add(1000000, 2000000) == 3000000\n\ndef test_add_negative_floats():\n    assert add(-1.5, -2.5) == -4.0\n\n\n# Test cases for subtract function\ndef test_subtract_positive_integers():\n    assert subtract(5, 2) == 3\n\ndef test_subtract_negative_integers():\n    as

In [26]:
import json

import json
import re

ai_output = response.text

# Extract JSON block
json_match = re.search(r'\{[\s\S]*\}', ai_output)

if not json_match:
    raise ValueError("No JSON object found in AI output")

json_str = json_match.group(0)

# Parse JSON
data = json.loads(json_str)

# Extract contents
test_file_name = data["test_file_name"]
test_part = data["test_file_content"]
readme_part = data["readme_content"]

print("✅ Files extracted from AI output")



✅ Files extracted from AI output


In [27]:
import base64
def push_file_to_github(repo, owner, path, content, message):
    url = f"https://api.github.com/repos/{owner}/{repo}/contents/{path}"

    encoded_content = base64.b64encode(content.encode()).decode()

    data = {
        "message": message,
        "content": encoded_content
    }

    headers = {
        "Authorization": f"token {GITHUB_TOKEN}",
        "Accept": "application/vnd.github.v3+json"
    }

    response = requests.put(url, json=data, headers=headers)

    if response.status_code in [200, 201]:
        print(f"✅ {path} pushed successfully")
    else:
        print(f"❌ Failed to push {path}")
        print(response.json())


In [28]:
push_file_to_github(
    repo=REPOSITORY_NAME,
    owner=GITHUB_USERNAME,
    path="test_math_utils.py",
    content=test_part,
    message="Add pytest tests generated by AI"
)

✅ test_math_utils.py pushed successfully


In [29]:
push_file_to_github(
    repo=REPOSITORY_NAME,
    owner=GITHUB_USERNAME,
    path="README.md",
    content=readme_part,
    message="Add README generated by AI"
)


✅ README.md pushed successfully
